<a href="https://colab.research.google.com/github/MichalGinat/UnitPlus/blob/main/unit%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install cudnn
!nvidia-smi

import torch
torch.cuda.is_available()
import tensorflow as tf
tf.test.gpu_device_name()

!nvcc --version

# install cuda
!pip install cuda-python==11.7.0

Sat Feb 24 13:01:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# clone repo and install other packages
!git clone https://github.com/algolzw/image-restoration-sde

!sed -i 's/==.*$//' image-restoration-sde/requirements.txt


# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install kaleido python-multipart cohere openai tiktoken fastapi uvicorn uvicorn ema_pytorch

!cd image-restoration-sde && pip install -r requirements.txt

!wget http://www.icst.pku.edu.cn/struct/att/Rain100H.zip
!unzip Rain100H.zip

Cloning into 'image-restoration-sde'...
remote: Enumerating objects: 699, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 699 (delta 79), reused 60 (delta 60), pack-reused 599
Receiving objects: 100% (699/699), 48.36 MiB | 25.50 MiB/s, done.
Resolving deltas: 100% (245/245), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.9 MB/s eta 0:00:00
     ━

In [ ]:
# set the config paths

!sed -i 's|dataroot_GT: datasets/rain/testH/GT|dataroot_GT: /content/Rain100H|' /content/image-restoration-sde/codes/config/deraining/options/test/ir-sde.yml
!sed -i 's|dataroot_LQ: datasets/rain/testH/LQ|dataroot_LQ: /content/Rain100H|' /content/image-restoration-sde/codes/config/deraining/options/test/ir-sde.yml

# /content/Rain100H <- location of the testing set
# /content/image-restoration-sde/codes/config/deraining/options/test <- location if the config .yml file

In [ ]:
# fetch the pre-trained model and place it at the correct destination

!wget 'https://drive.usercontent.google.com/download?id=1o6ELATcKOw96Uno8rJVB20vcLRWWBnu2&export=download&authuser=0&confirm=t&uuid=0794783f-5d4e-47e7-97ed-04aba46bb6d2&at=APZUnTVYyBgi6WvNNpASArspyfWE:1707728056992'
!mv 'download?id=1o6ELATcKOw96Uno8rJVB20vcLRWWBnu2&export=download&authuser=0&confirm=t&uuid=0794783f-5d4e-47e7-97ed-04aba46bb6d2&at=APZUnTVYyBgi6WvNNpASArspyfWE:1707728056992' rain100h_sde.pth
!mkdir /content/image-restoration-sde/codes/config/deraining/pretrained/
!cp rain100h_sde.pth /content/image-restoration-sde/codes/config/deraining/pretrained/

--2024-02-24 13:02:45--  https://drive.usercontent.google.com/download?id=1o6ELATcKOw96Uno8rJVB20vcLRWWBnu2&export=download&authuser=0&confirm=t&uuid=0794783f-5d4e-47e7-97ed-04aba46bb6d2&at=APZUnTVYyBgi6WvNNpASArspyfWE:1707728056992
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.128.132, 2a00:1450:4013:c02::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.128.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548648748 (523M) [application/octet-stream]
Saving to: ‘download?id=1o6ELATcKOw96Uno8rJVB20vcLRWWBnu2&export=download&authuser=0&confirm=t&uuid=0794783f-5d4e-47e7-97ed-04aba46bb6d2&at=APZUnTVYyBgi6WvNNpASArspyfWE:1707728056992’

download?id=1o6ELAT 100%[===================>] 523.23M   163MB/s    in 3.2s    

2024-02-24 13:02:49 (163 MB/s) - ‘download?id=1o6ELATcKOw96Uno8rJVB20vcLRWWBnu2&export=download&authuser=0&confirm=t&uuid=0794783f-5d4e-47e7-97ed-04aba46bb6d2&at=APZUnTVYyBgi6WvNNpAS

In [ ]:
# Start the testing
!cd /content/image-restoration-sde/codes/config/deraining && python3 test.py -opt=options/test/ir-sde.yml

export CUDA_VISIBLE_DEVICES=0
OrderedDict([('name', 'Val_Dataset'), ('mode', 'LQGT'), ('dataroot_GT', '/content/Rain100H'), ('dataroot_LQ', '/content/Rain100H')])
rm: cannot remove './result': No such file or directory
24-02-24 13:02:56.818 - INFO:   name: ir-sde
  suffix: None
  model: denoising
  distortion: derain
  gpu_ids: [0]
  sde:[
    max_sigma: 10
    T: 100
    schedule: cosine
    eps: 0.005
  ]
  degradation:[
    sigma: 25
    noise_type: G
    scale: 4
  ]
  datasets:[
    test1:[
      name: Val_Dataset
      mode: LQGT
      dataroot_GT: /content/Rain100H
      dataroot_LQ: /content/Rain100H
      phase: test1
      scale: 1
      data_type: img
    ]
  ]
  network_G:[
    which_model_G: ConditionalUNet
    setting:[
      in_nc: 3
      out_nc: 3
      nf: 64
      depth: 4
    ]
  ]
  path:[
    pretrain_model_G: pretrained/rain100h_sde.pth
    root: /content/image-restoration-sde
    results_root: /content/image-restoration-sde/results/deraining/ir-sde
    log: /con

In [ ]:
# just a useful shell for the computing instance YOU DON"T NEED TO RUN THAT FOR THE MODEL
!pip install google-colab-shell

from google_colab_shell import getshell
getshell(500)

  Preparing metadata (setup.py) ... done
  Created wheel for google-colab-shell: filename=google_colab_shell-0.2-py3-none-any.whl size=4107 sha256=49e4b1303526f11965efe080a30377f31a20797bc7d1c597e05d78a3e83e3df8
  Stored in directory: /root/.cache/pip/wheels/f9/36/65/95dd4599be065418a9fe1f482674c8e716ce540f3f484681d2
Successfully built google-colab-shell
